### 0) Data Importing and Helper Function Definition:

In [ ]:
import numpy as np

DATA_folder  = '../../Data/'
data = np.load(DATA_folder+'train_imgs.npy')/1.0
lbls = np.load(DATA_folder+'train_lbls.npy')
test_data = np.load(DATA_folder+'test_imgs.npy')/1.0
test_lbls = np.load(DATA_folder+'test_lbls.npy')

In [ ]:
#data[lbls==0,:].shape

### 1) Definition of RBF Transformer Object:

RBF Transformer is a scikit-learn compatible transformer object that implements:

    - fit method       - clusters the digit-separated data, and computes cluster centers and inv. sq. deviations
    - transform method - based on obtained centers and deviations it computes the fi values (RBF layer outputs) as 
                         simple Gaussian functions -> Fi_k(x) = exp{ -sqrt( sum[ (x_i-c_k_i)/(dev_i^2) ] ) } 
                         
## Normalizing each Fi row with row sum -> STABILIZED the Regression a lot!!!

#### Definition of Clustering Function:

In [ ]:
### Clustering:

from sklearn.cluster import KMeans

# Function for data clustering, and computation of cluster center vectors and inv. sq. deviation vectors
def form_clusters(data, n_kmeans):
    
    n_clst = min(n_kmeans,data.shape[0])
    
    kmeans = KMeans(n_clusters=n_clst, random_state=0, init='k-means++', algorithm='elkan')
    lbls_kmeans = kmeans.fit_predict(data)
    
    lbls_set = [lbls_kmeans] # just as an option to include more clustering methods
    
    centers   = []
    inv_sq_devs  = []
    
    # Find cluster centers and covar matrix:
    for lbls in lbls_set:
        for k in range(max(lbls)+1):
            
            cluster = data[lbls==k,:]
            centers.append(cluster.mean(axis=0))
            dev = np.std(cluster,axis=0)
            dev[dev==0]+=1e-3 # to avoid Infs and NaNs
            inv_sq_devs.append(np.reciprocal(np.square(dev)))
            
        del cluster, dev
    return centers, inv_sq_devs;


#### Definition of RBF (Fi) Function:

In [ ]:
# Function to compute whole Fi output for given dataset (for all RB centers)
def fi_transform(data, all_centers, all_inv_sq_devs):
    # data        - given dataset matrix for which to compute fi values
    # center      - list of all center vectors on which to compute fi vals
    # inv_sq_devs - list of all reciprocal sq. deviation vectors on which to compute fi vals     

    new_data = np.empty((data.shape[0],len(all_centers)))
    st = 0
    ch = 200
    ns = data.shape[0]
    # Process data in chunks of 200 smpls (optimal speed)
    while st<ns:
        en = min(st+ch,ns)
        for k in range(len(all_centers)):
            # new_data[st:en,k] = calc_fi_column(data[st:en], all_centers[k], all_inv_sq_devs[k])
            new_data[st:en,k] = np.dot(np.square(data[st:en] - np.repeat(all_centers[k][np.newaxis,:],data[st:en].shape[0],axis=0)),all_inv_sq_devs[k])
        st = en
    new_data = np.exp(-np.sqrt(new_data))
    return new_data

#### Definition of RBF Transformer (sklearn compatible object):

In [ ]:
# http://scikit-learn.org/stable/modules/classes.html#module-sklearn.base

from sklearn.base import BaseEstimator, TransformerMixin

# SKLEARN Compatible Transformer - supports fit method (custering data) and transform method (calculating fi values)
class myRBFtransformer(BaseEstimator, TransformerMixin):
    
    # Transformer initialization (default to 50 kMeans clusters)
    def __init__(self, kmeans_ratio=0.5, n_fi_max=10):
        self.kmeans_ratio = kmeans_ratio # ratio used to calc the number of clusters (n=r*smpls/10)
        self.n_fi_max     = n_fi_max     # num of max fi vals to return per digit
        self.centers      = []           # list of cluster center vectors
        self.inv_sq_devs  = []           # list of cluster inv. sq. dev. vectors
        self.num_centers  = []           # list of number of centers per digit
        # print(self.kmeans_ratio,self.n_fi_max)
    
    # Clusters each digit and finds cluster centers and deviations:
    def fit(self, X, y):
        
        self.centers     = []
        self.inv_sq_devs = []
        self.num_centers = []
        
        # Calc. num. of clusters per digit based on assigned ratio (ratio*(num of smpls / 10))
        n_kmeans = max(round(self.kmeans_ratio*X.shape[0]/10), 1)
        # print(n_kmeans)
        
        # Cluster the data over each digit
        for dig in range(10):
            # print('Clustering digit: ',dig)
            data = X[y==dig,:]
            centers, inv_sq_devs = form_clusters(data, n_kmeans)
            self.centers.extend(centers)
            self.inv_sq_devs.extend(inv_sq_devs)
            self.num_centers.append(len(centers))
        
        # Limit the n_fi_max to the min. number of centers per digit 
        self.n_fi_max = min(self.n_fi_max,min(self.num_centers))
        
        return self
    
    # Computes fi values (with Gaussian function) based on obtained cluster centers and deviations
    def transform(self, X, y=None):
        
        # Compute all Fi values:
        # print('- Calculating all fi outputs !!!')
        all_fis = fi_transform(X, self.centers, self.inv_sq_devs)
        
        # Find n max fi vals over each digit:
        # print('- Selecting n max fi outputs !!!')
        n_max  = self.n_fi_max
        result = np.empty((X.shape[0],n_max*10))
        start  = 0
        for k in range(10):
            end = start + self.num_centers[k]
            result[:,(k*n_max):((k+1)*n_max)] = np.sort(all_fis[:,start:end],axis=1)[:,-n_max:]
            start = end
        del all_fis
        
        # Normalize each row with the sum of that row's elements:
        # result = np.divide(result,np.sum(result,axis=1).reshape((result.shape[0],1)))
        row_sum = np.sum(result,axis=1).reshape((result.shape[0],1))
        row_sum[row_sum==0]=1e-12
        result = np.divide(result,row_sum)
        # print('Transform finished !!!')
        
        return result

In [ ]:
# # Test:
# rbf = myRBFtransformer(n_kmeans = 10)
# a   = rbf.fit_transform(X=data[:500,:],y=lbls[:500])
# a.max(axis=1)

### 3) Pipelining the Model:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model  import LogisticRegression

n_pca        = 545 # First dimension to retain 99 % variance (stanford edu recomendation for images)!!!
kmeans_ratio = 1
n_fi_max     = 10

scaler = StandardScaler()
pca    = PCA(n_components=n_pca, whiten=True)
rbf    = myRBFtransformer(kmeans_ratio=kmeans_ratio,n_fi_max=n_fi_max)
logreg = LogisticRegression(tol=1e-12,C=1e+12,random_state=12,solver='liblinear')


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
my_pipe = Pipeline(steps=[('scal1', scaler), ('pca', pca), ('rbf',rbf), ('logreg',logreg)])

In [ ]:
pred_lbls = my_pipe.predict(test_data)

In [ ]:
sum(pred_lbls==test_lbls)/len(test_lbls)

In [ ]:
# from sklearn.model_selection import GridSearchCV
from dask_searchcv import GridSearchCV

Ks = [0.1, 0.3, 0.5, 0.8]
Ns = [3, 7, 10, 15]
Cs = [1e3, 1e6, 1e9, 1e12]

param_grid = [
    {
        'rbf__kmeans_ratio': Ks,
        'rbf__n_fi_max':     Ns,
        'logreg__C':         Cs
    }
]


In [ ]:
grid = GridSearchCV(my_pipe, cv=3, n_jobs=-1, param_grid=param_grid)

In [ ]:
grid.fit(test_data[:], test_lbls[:])

In [ ]:
print(grid.best_params_)

In [ ]:
# ### For this grid
# Ks = [0.1, 0.3, 0.5, 0.8]
# Ns = [3, 7, 10, 15]
# Cs = [1e3, 1e6, 1e9, 1e12]
# # Best set was
# K = 0.1
# N = 15
# C = 1e9
# # at 71.79%


In [ ]:
max(grid.cv_results_['mean_test_score'])

In [ ]:
grid.cv_results_['std_test_score']